In [76]:
import pandas as pd
import geopandas as gpd
import matplotlib as plt
import numpy as np

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_columns', 500)

In [3]:
new_data = pd.read_stata('/Users/danielgrenell/Documents/Erica Vladimer Data/12cdenhancedupdate1019_clean.dta')

In [78]:
df = new_data[['precinct', 'voters_age', 'voters_birthdate', 'primary_20180626', 'primary_20180913', 'special_20190226', 'primary_20190625']]

In [79]:
df['under_30'] = df.voters_age.apply(lambda x: 1 if x<30 else 0)
df['under_60'] = df.voters_age.apply(lambda x: 1 if x<60 else 0)

In [80]:
elections = ['special_20190226', 'primary_20190625', 'primary_20180913', 'primary_20180626']
# look at 6-25 multiple columns to see if there's a difference
# add columns for 2 and not 1
# add column for first vote in 6-25
# 4x voters
# final data frame -- rows will be election districts, columns = total voters from other spreadsheet, number voters
# in all 4 elections, number under 30 voters, number under 60 voters, average age in each election, 

In [81]:
for election in elections:
    df[election] = df[election].apply(lambda x: 1 if x=='Y' else 0)
    df[election + "date"] = pd.to_datetime(election[-8:], format='%Y%m%d')

In [82]:
df['KEY'] = df.precinct.apply(lambda x: int(x[-2:] + x[-6:-3]))

In [83]:
df['voters_birthdate'] = pd.to_datetime(df['voters_birthdate'])

In [84]:
for election in elections:
    df["age_in_"+election] = round((df[election+'date']-df['voters_birthdate']).astype(int)/(60*60*24*365*1000000000),2)

In [85]:
df['four_x'] = df[elections[0]]*df[elections[1]]*df[elections[2]]*df[elections[3]]

In [86]:
df['skipped_patel'] = ((df[elections[2]]==1) & (df[elections[3]]==0)).astype(int)

In [87]:
df['new_voter'] = ((df[elections[1]]==1) & (df[elections[0]]==0) & (df[elections[2]]==0) & (df[elections[3]]==0)).astype(int)

In [88]:
df['four_x_under_30'] = df.four_x * df.under_30
df['four_x_under_60'] = df.four_x * df.under_60

In [89]:
df.drop(['precinct', 'voters_age', 'voters_birthdate', 'special_20190226date', 'primary_20190625date', 'primary_20180913date', 'primary_20180626date'], axis=1, inplace=True)

In [90]:
df.rename(columns={"primary_20180626": "06-26-2018", "primary_20180913": "09-13-2018", "special_20190226": "02-26-2019", 'primary_20190625': '06-25-2019', "age_in_primary_20180626": "age_in_06-26-2018", "age_in_primary_20180913": "age_in_09-13-2018", "age_in_special_20190226": "age_in_02-26-2019", 'age_in_primary_20190625': 'age_in_06-25-2019'}, inplace=True)

In [91]:
cols = ['KEY', '06-26-2018', '09-13-2018', '02-26-2019', '06-25-2019',
        'age_in_02-26-2019', 'age_in_06-25-2019', 'age_in_09-13-2018', 'age_in_06-26-2018',
        'under_30', 'under_60', 'new_voter', 'four_x', 'four_x_under_30', 'four_x_under_60', 'skipped_patel']
df = df[cols]

In [92]:
dates = ['06-26-2018', '09-13-2018', '02-26-2019', '06-25-2019']
for date in dates:
    df['age_in_' + date] = df[date] * df['age_in_' + date]
    df['age_in_' + date].replace({0: np.NaN}, inplace=True)

In [93]:
sum_group = ['KEY', '06-26-2018', '09-13-2018', '02-26-2019', '06-25-2019', 'under_30', 'under_60', 'new_voter', 'four_x', 'four_x_under_30', 'four_x_under_60', 'skipped_patel']
avg_group = ['KEY', 'age_in_02-26-2019', 'age_in_06-25-2019', 'age_in_09-13-2018', 'age_in_06-26-2018']
sum_df = df[sum_group].groupby(['KEY']).sum()
avg_df = df[avg_group].groupby(['KEY']).mean()

In [94]:
avg_df

,age_in_02-26-2019,age_in_06-25-2019,age_in_09-13-2018,age_in_06-26-2018
KEY,,,,
30038,46.369846,43.097529,44.653832,42.559231
30056,46.133276,44.235566,43.849700,44.379500
30064,43.152222,42.917321,43.243750,43.057000
30066,43.935857,43.635172,44.648073,42.914474
30067,42.906364,43.011558,46.532771,43.430862
30068,46.276714,42.434567,44.319167,43.722838
30069,52.388070,47.222100,49.561933,47.933864
30078,41.180513,41.339206,40.511786,38.030400
30079,44.237538,40.900933,41.653936,40.090833


In [95]:
df = sum_df.merge(avg_df, on='KEY')

In [96]:
df.to_csv('Four election data by ED.csv')